In [1]:
import pandas as pd
import json

In [26]:
def resetDict():
    # instantiate dictionary for dataframe
    games = dict()
    games["gameId"] = list()
    games["teamId"] = list()
    games["playerId"] = list()
    games["min"] = list()
    games["fgm"] = list()
    games["fga"] = list()
    games["ftm"] = list()
    games["fta"] = list()
    games["tpm"] = list()
    games["tpa"] = list()
    games["offReb"] = list()
    games["defReb"] = list()
    games["assists"] = list()
    games["steals"] = list()
    games["blocks"] = list()
    games["pFouls"] = list()
    games["turnovers"] = list()
    games["plusMinus"] = list()
#     print("resetDict")
    return games

def fillDataFrame(games, json_file):
    for raw_data in json_file:
        dat = raw_data["api"]["statistics"]
        for d in dat:
            games["gameId"].append(d["gameId"])
            games["teamId"].append(d["teamId"])
            games["playerId"].append(d["playerId"])
            games["min"].append(d["min"])
            games["fgm"].append(d["fgm"])
            games["fga"].append(d["fga"])
            games["ftm"].append(d["ftm"])
            games["fta"].append(d["fta"])
            games["tpm"].append(d["tpm"])
            games["tpa"].append(d["tpa"])
            games["offReb"].append(d["offReb"])
            games["defReb"].append(d["defReb"])
            games["assists"].append(d["assists"])
            games["steals"].append(d["steals"])
            games["blocks"].append(d["blocks"])
            games["pFouls"].append(d["pFouls"])
            games["turnovers"].append(d["turnovers"])
            games["plusMinus"].append(d["plusMinus"])
    
    df = pd.DataFrame(games)
#     print(df.dtypes)
    return df

def toIntMin(pStr):
    pStr = pStr.strip()
    if pStr == "":
        return 0
    elif len(pStr) < 1:
        return 0
    
    col_idx = pStr.find(":")
    ret = int(pStr[:col_idx])
    return ret

def toIntSec(pStr):
    pStr = pStr.strip()
    if pStr == "":
        return 0
    elif len(pStr) < 1:
        return 0
    
    col_idx = pStr.find(":")
    ret = int(pStr[col_idx+1:])
    return ret

def convertTodigit(digit, isInt=True):
    digit = digit.strip()
    if digit == "":
        return 0
    elif len(digit) < 1:
        return 0
    
    if isInt:
        return int(digit)
    else:
        return float(digit)

def convertDFdatatypes(df):
    df["minutes"] = df["min"].apply(toIntMin)
    df["seconds"] = df["min"].apply(toIntSec)
    df["fgm"] = df["fgm"].apply(convertTodigit)
    df["fga"] = df["fga"].apply(convertTodigit)
    df["ftm"] = df["ftm"].apply(convertTodigit)
    df["fta"] = df["fta"].apply(convertTodigit)
    df["tpm"] = df["tpm"].apply(convertTodigit)
    df["tpa"] = df["tpa"].apply(convertTodigit)
    df["offReb"] = df["offReb"].apply(convertTodigit)
    df["defReb"] = df["defReb"].apply(convertTodigit)
    df["assists"] = df["assists"].apply(convertTodigit)
    df["steals"] = df["steals"].apply(convertTodigit)
    df["blocks"] = df["blocks"].apply(convertTodigit)
    df["pFouls"] = df["pFouls"].apply(convertTodigit)
    df["turnovers"] = df["turnovers"].apply(convertTodigit)
    df["plusMinus"] = df["plusMinus"].apply(convertTodigit)
    
    return df

In [3]:
def process(file_name):
    with open(file_name) as json_file:
        data = json.load(json_file)
    
    games = resetDict()
    df = fillDataFrame(games, data)
    df = convertDFdatatypes(df)
    return df

In [29]:
df = process('game_details.json')
df2 = process('game_details_2.json')

In [33]:
df.append(df2, ignore_index=True)

,gameId,teamId,playerId,min,fgm,fga,ftm,fta,tpm,tpa,offReb,defReb,assists,steals,blocks,pFouls,turnovers,plusMinus,minutes,seconds
0,4308,18,688,24:44,1,8,2,3,0,0,1,2,1,4,0,2,1,-16,24,44
1,4308,18,1097,31:59,4,9,2,2,3,6,1,4,0,1,0,5,4,-9,31,59
2,4308,18,1099,31:28,8,13,3,3,0,0,3,10,1,0,0,3,1,-14,31,28
3,4308,18,1100,19:58,6,12,0,0,1,6,0,1,2,0,0,1,2,-12,19,58
4,4308,18,706,39:29,6,22,4,4,3,12,0,3,5,4,0,4,5,-12,39,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23136,5207,8,219,16:30,1,4,0,0,1,3,0,1,3,0,1,2,1,1,16,30
23137,5207,8,943,18:37,2,6,0,0,2,5,0,5,2,0,0,2,0,1,18,37
23138,5207,8,311,4:23,0,1,0,0,0,1,0,1,1,0,0,0,0,1,4,23
23139,5207,8,806,4:23,1,3,0,0,1,1,0,0,0,0,0,0,0,1,4,23


In [37]:
df["teamId"].value_counts().sort_index().shape

1     443
10    419
11    422
12     12
13     32
14    375
15    431
16    462
17    459
18     27
19    436
2     428
20    440
21    412
22    436
23    490
24    456
25    396
26    452
27    425
28    441
29    435
30    437
31    434
32     15
33     12
38    464
4     418
40    461
41    445
5     455
6     416
7     399
8     395
87     13
9     414
90     12
91     20
92     11
93     31
Name: teamId, dtype: int64